In [1]:
from matplotlib import pyplot as plt
from rnaseq3 import RNAseq as RNAseq3
from os.path import expanduser as exusr
from CellModels.Cells.IO import CellReader
from CellModels.Clustering.IO import ClusteringReader
import numpy as np
import pandas as pd
import logging

%load_ext autoreload
%autoreload 2

In [2]:
aerts_seq = RNAseq3(exusr('~/Google Drive File Stream/My Drive/Projects/RDN-WDP/contrib/scRNAseq/EyeAntennal_Combined_DG2_v4.csv'))
ariss_seq = RNAseq3(exusr('~/Google Drive File Stream/My Drive/Projects/RDN-WDP/contrib/GSE115476_RAW/GSM3178869_DMS.expr.txt'))
aerts = aerts_seq.data
ariss = ariss_seq.data


In [3]:
data = CellReader.read(exusr('~/Google Drive File Stream/My Drive/Projects/RDN-WDP/processing/samples_complete.csv'))
data

Position                                          \
                          Raw                   Normalized              
                            x        y        z          x          y   
Gene Sample Nucleus                                                     
Abl  1Q8GA8 1           2.779  359.282   16.923   0.111127        NaN   
            2          36.385  433.355   18.801   1.454965  15.329020   
            3          49.492  446.095   18.521   1.979088  15.838467   
            4          74.157  469.307   22.554   2.965394        NaN   
            5         152.613  531.957   21.376   6.102696  19.271921   
...                       ...      ...      ...        ...        ...   
NaN  vTdokD 10752    2041.602  159.832  125.421  77.244817        NaN   
            10753    1556.359  188.648  125.377  58.885457  -9.862428   
            10754    1572.244  238.565  125.243  59.486472  -7.973799   
            10755    1999.442  293.631  125.116  75.649677  -4.890355   
            10756    1921.119  346.511  125.235  72.686296  -2.889620   

                                         Measurements                  \
                                  Scaled          Raw                   
                            z          y       Volume   Venus mCherry   
Gene Sample Nucleus                                                     
Abl  1Q8GA8 1        0.676718  14.366985         1773  18.146  28.548   
            2        0.751815  17.329020         5020   8.886  43.103   
            3        0.740619  17.838467         4522   9.722  31.826   
            4        0.901890  18.766670        15872   9.787  19.014   
            5        0.854784  21.271921        12339  10.121  25.868   
...                       ...        ...          ...     ...     ...   
NaN  vTdokD 10752    4.745353   6.047307         4581  42.420   4.710   
            10753    4.743688   7.137572         7902   8.976   3.379   
            10754    4.738618   9.026201         8378   9.282   3.024   
            10755    4.733813  11.109645        11080  18.370   3.218   
            10756    4.738316  13.110380         6274  14.035   3.180   

                                                                          \
                              Normalized                      Prominence   
                         DAPI    mCherry     Venus       DAPI    mCherry   
Gene Sample Nucleus                                                        
Abl  1Q8GA8 1         997.067   0.349663  0.550104  19.212920        NaN   
            2         718.966   0.171228  0.830571  13.854070   0.837475   
            3         747.146   0.187337  0.613269  14.397083   1.006505   
            4         921.009   0.188590  0.366389  17.747325        NaN   
            5        1099.488   0.195026  0.498462  21.186515   1.047569   
...                       ...        ...       ...        ...        ...   
NaN  vTdokD 10752    1294.884   1.212765  0.134656  37.020027        NaN   
            10753    1258.350   0.256619  0.096604  35.975540   0.795890   
            10754    1243.135   0.265367  0.086455  35.540552   0.789268   
            10755    1137.998   0.525188  0.092001  32.534742   0.865798   
            10756    1068.524   0.401253  0.090914  30.548519   0.856474   

                                                       
                                    Angle              
                        Venus     mCherry       Venus  
Gene Sample Nucleus                                    
Abl  1Q8GA8 1             NaN         NaN         NaN  
            2        0.910950   72.597306   88.679193  
            3        0.591404   44.716713   59.522732  
            4             NaN         NaN         NaN  
            5        0.808834  173.828641  109.918618  
...                       ...         ...         ...  
NaN  vTdokD 10752         NaN         NaN         NaN  
            10753    1.113405   39.735418  166.383565  
            1075

In [4]:
clustered = ClusteringReader.read(exusr('~/Google Drive File Stream/My Drive/Projects/RDN-WDP/processing/clustering/bigc100k6n20r1000_metadata.yml'))
clustered.cells


Position                      Measurements            \
                    Normalized                        Normalized             
                             x          y         z      mCherry     Venus   
Gene Sample Nucleus                                                          
Abl  1Q8GA8 1239543   1.454965  15.329020  0.751815     0.171228  0.830571   
            1239544   1.979088  15.838467  0.740619     0.187337  0.613269   
            1239545   6.102696  19.271921  0.854784     0.195026  0.498462   
            1239546  10.614307  22.123290  0.778087     0.162865  0.292009   
            1239547   0.639369  12.605514  0.797042     0.245897  0.775441   
...                        ...        ...       ...          ...       ...   
spdo U0R0WS 1989145  13.636071  -0.953809  4.117531     0.239002  0.299819   
            1989146  17.377004   0.434604  4.159967     0.320409  0.350425   
            1989147  15.133310   3.362016  4.137667     0.206911  0.285008   
            1989148  13.815633  -3.539903  4.202985     0.192044  0.313957   
            1989149  11.240356  -2.595631  4.162754     0.202423  0.247699   

                                                                        \
                                  Raw Prominence                 Angle   
                          DAPI Volume    mCherry     Venus     mCherry   
Gene Sample Nucleus                                                      
Abl  1Q8GA8 1239543  13.854070   5020   0.837475  0.910950   72.597306   
            1239544  14.397083   4522   1.006505  0.591404   44.716713   
            1239545  21.186515  12339   1.047569  0.808834  173.828641   
            1239546  18.692125   8100   0.861932  0.327443  129.476525   
            1239547  19.484099   4806   1.123186  0.985807   37.003297   
...                        ...    ...        ...       ...         ...   
spdo U0R0WS 1989145  19.406605   8595   0.570954  1.257708    7.767949   
            1989146  18.459796   7089   0.479263  1.425268   20.500289   
            1989147  19.834397   7318   0.416987  1.138248  162.220803   
            1989148  18.634784   4006   0.580034  1.377011   64.370008   
            1989149  23.191932   3773   0.573739  1.093847   29.855944   

                                Cluster  
                                   ward  
                          Venus       6  
Gene Sample Nucleus                      
Abl  1Q8GA8 1239543   88.679193     4.0  
            1239544   59.522732     4.0  
            1239545  109.918618     4.0  
            1239546   51.013798     4.0  
            1239547   88.526588     4.0  
...                         ...     ...  
spdo U0R0WS 1989145   14.861582     5.0  
            1989146  152.308561     3.0  
            1989147  163.978249     3.0  
            1989148   12.571874     5.0  
            1989149    4.547415     5.0  

[2368709 rows x 12 columns]